In [32]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from factor_analyzer.factor_analyzer import calculate_kmo

In [37]:
origin=read_excel('행정동병합테스트22(반포본동).xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,108,626,110,956.51,3,1.000000,0,16,0,5
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,385,1814,311,0.00,3,0.000000,10,34,0,5
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1033,3495,491,1.14,3,0.001192,10,54,0,5


In [38]:
origin.isna().sum()
origin1=origin.fillna(0)
scaler=MinMaxScaler()
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수'],
      dtype='object')

In [39]:
df = origin1.copy()
df['면적minmax']=scaler.fit_transform(df[['면적 (k㎡)']])
df['공공체육시설minmax']=scaler.fit_transform(df[['공공체육시설_수']])
df['문화시설minmax']=scaler.fit_transform(df[['문화시설_수']])
df['유통업체minmax']=scaler.fit_transform(df[['행정동별유통업체수']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,4,0.352898,0,32,0,3,0.187952,0.256410,0.055556,0.0
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,0.063554,0,27,0,12,0.080321,0.205128,0.120370,0.0
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,3,1.000000,0,16,0,5,0.101205,0.051282,0.287037,0.0
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,3,0.000000,10,34,0,5,0.163855,0.128205,0.046296,0.0
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3,0.001192,10,54,0,5,0.693976,0.128205,0.101852,0.0


In [40]:
df['여가생활지수'] = (df['공공체육시설minmax'] + df['문화시설minmax'] + df['유통업체minmax']) / df['면적minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,0.187952,0.256410,0.055556,0.0,1.659818
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,0.080321,0.205128,0.120370,0.0,4.052457
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,0.101205,0.051282,0.287037,0.0,3.342915
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,0.163855,0.128205,0.046296,0.0,1.064972
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,0.693976,0.128205,0.101852,0.0,0.331506


In [41]:
df['여가생활지수'].value_counts().sort_index()

여가생활지수
0.000000     37
0.050560      1
0.060006      1
0.067414      1
0.103854      1
             ..
23.343750     1
24.958042     1
25.525794     1
43.340011     1
inf           1
Name: count, Length: 378, dtype: int64

In [42]:
df['여가생활지수'].isna().sum()

1

In [43]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '면적minmax', '공공체육시설minmax', '문화시설minmax', '유통업체minmax',
       '여가생활지수'],
      dtype='object')

In [44]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','면적 (k㎡)','공공체육시설수','문화시설수','행정동별유통업체수','면적minmax','공공체육시설minmax','문화시설minmax','유통업체minmax','여가생활지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),행정동별유통업체수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,2.57,0,0.187952,0.256410,0.055556,0.0,1.659818
1,11110,종로구,1111053000,사직동,1.23,0,0.080321,0.205128,0.120370,0.0,4.052457
2,11110,종로구,1111054000,삼청동,1.49,0,0.101205,0.051282,0.287037,0.0,3.342915
3,11110,종로구,1111055000,부암동,2.27,0,0.163855,0.128205,0.046296,0.0,1.064972
4,11110,종로구,1111056000,평창동,8.87,0,0.693976,0.128205,0.101852,0.0,0.331506


In [45]:
df1

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),행정동별유통업체수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,2.57,0,0.187952,0.256410,0.055556,0.0000,1.659818
1,11110,종로구,1111053000,사직동,1.23,0,0.080321,0.205128,0.120370,0.0000,4.052457
2,11110,종로구,1111054000,삼청동,1.49,0,0.101205,0.051282,0.287037,0.0000,3.342915
3,11110,종로구,1111055000,부암동,2.27,0,0.163855,0.128205,0.046296,0.0000,1.064972
4,11110,종로구,1111056000,평창동,8.87,0,0.693976,0.128205,0.101852,0.0000,0.331506
...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,0.67,1,0.035341,0.000000,0.018519,0.0625,2.292456
422,11740,강동구,1174066000,성내제3동,0.71,1,0.038554,0.000000,0.009259,0.0625,1.861256
423,11740,강동구,1174068500,길동,1.61,2,0.110843,0.025641,0.009259,0.1250,1.442579
424,11740,강동구,1174069000,둔촌제1동,0.92,0,0.055422,0.000000,0.000000,0.0000,0.000000


In [46]:
df2=df1.fillna(0)
df2

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),행정동별유통업체수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,2.57,0,0.187952,0.256410,0.055556,0.0000,1.659818
1,11110,종로구,1111053000,사직동,1.23,0,0.080321,0.205128,0.120370,0.0000,4.052457
2,11110,종로구,1111054000,삼청동,1.49,0,0.101205,0.051282,0.287037,0.0000,3.342915
3,11110,종로구,1111055000,부암동,2.27,0,0.163855,0.128205,0.046296,0.0000,1.064972
4,11110,종로구,1111056000,평창동,8.87,0,0.693976,0.128205,0.101852,0.0000,0.331506
...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,0.67,1,0.035341,0.000000,0.018519,0.0625,2.292456
422,11740,강동구,1174066000,성내제3동,0.71,1,0.038554,0.000000,0.009259,0.0625,1.861256
423,11740,강동구,1174068500,길동,1.61,2,0.110843,0.025641,0.009259,0.1250,1.442579
424,11740,강동구,1174069000,둔촌제1동,0.92,0,0.055422,0.000000,0.000000,0.0000,0.000000


In [47]:
df2['여가생활지수'].value_counts().sort_index()

여가생활지수
0.000000     38
0.050560      1
0.060006      1
0.067414      1
0.103854      1
             ..
23.343750     1
24.958042     1
25.525794     1
43.340011     1
inf           1
Name: count, Length: 378, dtype: int64

In [48]:
df2['여가생활지수'].replace([np.inf, -np.inf], 0, inplace=True)
df2

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),행정동별유통업체수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,2.57,0,0.187952,0.256410,0.055556,0.0000,1.659818
1,11110,종로구,1111053000,사직동,1.23,0,0.080321,0.205128,0.120370,0.0000,4.052457
2,11110,종로구,1111054000,삼청동,1.49,0,0.101205,0.051282,0.287037,0.0000,3.342915
3,11110,종로구,1111055000,부암동,2.27,0,0.163855,0.128205,0.046296,0.0000,1.064972
4,11110,종로구,1111056000,평창동,8.87,0,0.693976,0.128205,0.101852,0.0000,0.331506
...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,0.67,1,0.035341,0.000000,0.018519,0.0625,2.292456
422,11740,강동구,1174066000,성내제3동,0.71,1,0.038554,0.000000,0.009259,0.0625,1.861256
423,11740,강동구,1174068500,길동,1.61,2,0.110843,0.025641,0.009259,0.1250,1.442579
424,11740,강동구,1174069000,둔촌제1동,0.92,0,0.055422,0.000000,0.000000,0.0000,0.000000


In [49]:
df2['여가생활지수'].isna().sum()

0

In [50]:
df2['여가생활지수minmax']=scaler.fit_transform(df2[['여가생활지수']])

In [51]:
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),행정동별유통업체수,면적minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수,여가생활지수minmax
0,11110,종로구,1111051500,청운효자동,2.57,0,0.187952,0.256410,0.055556,0.0,1.659818,0.038298
1,11110,종로구,1111053000,사직동,1.23,0,0.080321,0.205128,0.120370,0.0,4.052457,0.093504
2,11110,종로구,1111054000,삼청동,1.49,0,0.101205,0.051282,0.287037,0.0,3.342915,0.077132
3,11110,종로구,1111055000,부암동,2.27,0,0.163855,0.128205,0.046296,0.0,1.064972,0.024572
4,11110,종로구,1111056000,평창동,8.87,0,0.693976,0.128205,0.101852,0.0,0.331506,0.007649


In [30]:
df2.to_excel('여가생활지수1.xlsx')